In [1]:

import os, sys, math, time
import numpy as np
import numpy.linalg as la
import plotly.graph_objects as go
import plotly.express as ex
from plotly.subplots import make_subplots
import pandas as pd

import json as js
import _pickle as pickle
import bz2
import ray

import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from collections import OrderedDict

sys.path.append("../")
import func

In [2]:
torch.cuda.device_count()

1

## Analyse features

In [9]:
data_path = "../../data/"
# load data
data = func.load(data_path+"LOCO_R2-default-locomotion.pbz2")
data_2 = func.load(data_path+"LOCO_R2-default-locomotion-small.pbz2")


In [7]:
class MLP(nn.Module):
    def __init__(self, dimensions:list, act_fn:str, keep_prob:float=.2, batch_size:int=1):
        super(MLP, self).__init__()
        self.dimensions = dimensions          #   [(in, h1), (h1, h2), ..., (hn, out)]
        self.act= act_fn                     #   func
        self.keep_prob = keep_prob          #   %
        self.batch_size = batch_size        #   int

        self.model = []

        assert(len(dimensions) >= 2)
        assert(batch_size > 0)
        assert(act_fn == "elu" or act_fn == "relu")
        assert(keep_prob < 1)
        for e in dimensions: assert(type(e) == int)

        self.build()
        self.model.apply(self.init_params)


    def build(self):
        layers = []
        for i, size in enumerate(zip(self.dimensions[0:], self.dimensions[1:])):
            layers.append(("fc"+str(i), nn.Linear(size[0], size[1])))
            if i < len(self.dimensions)-2:
                layers.append(("act"+str(i), self.activation(self.act)))
                layers.append(("drop"+str(i+1), nn.Dropout(self.keep_prob)))

        self.model = nn.Sequential(OrderedDict(layers))


    def forward(self, x:torch.Tensor) -> torch.Tensor:
        return self.model(x)

    @staticmethod
    def activation(fn_name):
        if fn_name == "elu":
            return nn.ELU()
        elif fn_name == "relu":
            return nn.ReLU()
        else:
            return nn.ReLU()

    @staticmethod
    def init_params(m):
        if type(m) == nn.Linear:
            nn.init.xavier_normal_(m.weight)
            m.bias.data.fill_(.01)

class LSTM(nn.Module):
    def __init__(self, input_size:int, hidden_size:int, num_layers:int = 1, keep_prob:float=.2, batch_size:int=1):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size          #   [(in, h1), (h1, h2), ..., (hn, out)]
        self.num_layers = num_layers
        self.keep_prob = keep_prob          #   %

        self.model = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, dropout=keep_prob)
        hidden_state = torch.randn(num_layers, batch_size, hidden_size)
        cell_state = torch.randn(num_layers, batch_size, hidden_size)
        self.hidden = (hidden_state, cell_state)

    def forward(self, x:torch.Tensor) -> torch.Tensor:
        h_t, h_n = self.model(x, self.hidden)
        self.hidden = h_n
        return h_t

class GRU(nn.Module):
    def __init__(self, input_size:int, hidden_size:int, num_layers:int = 1, keep_prob:float=.2, batch_size:int=1):
        super(GRU, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size          #   [(in, h1), (h1, h2), ..., (hn, out)]
        self.num_layers = num_layers
        self.keep_prob = keep_prob          #   %

        self.model = nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, dropout=keep_prob)
        hidden_state = torch.randn(num_layers, batch_size, hidden_size)
        cell_state = torch.randn(num_layers, batch_size, hidden_size)
        self.hidden = (hidden_state, cell_state)

    def forward(self, x:torch.Tensor) -> torch.Tensor:
        h_t, h_n = self.model(x, self.hidden)
        self.hidden = h_n
        return h_t


In [8]:
class DEC_AE(nn.Module):
    def __init__(self, encoder:nn.Module, decoder:nn.Module):
        super(DEC_AE, self).__init__()
        self.encoder = encoder
        self.decoder = decoder


    def forward(self, x):
        return self.decoder(self.encoder(x))

In [13]:
# Prepare train data
all_data = []
for d in data:
    d = pickle.loads(d)
    pos = []
    for f in d["frames"]:
        p = [jo["pos"] for jo in f]
        pos.append(p)
    all_data.append(pos)

input_data = np.array([np.concatenate([p for p in j]) for pos in all_data for j in pos])
print(input_data.shape)

(1440, 63)


In [16]:
data_ratio = (.7, .15, .15) # training, validation, testing
SEED = 2021
batch_size = 1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

x_tensor = torch.from_numpy(input_data).float()
y_tensor = torch.from_numpy(input_data).float()

dataset = TensorDataset(x_tensor, y_tensor)
N = len(dataset)

train_ratio = int(data_ratio[0]*N)
val_ratio = int(data_ratio[1] * N)
test_ratio = int(N-train_ratio-val_ratio)
print("Train: ", train_ratio, ", Validation: ", val_ratio, ", Test: ", test_ratio)

train_set, val_set, test_set = random_split(dataset, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(SEED))

train_loader = DataLoader(dataset=train_set, batch_size=batch_size)
val_loader = DataLoader(dataset=val_set, batch_size=batch_size)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size)


cuda
Train:  1007 , Validation:  216 , Test:  217


In [21]:
# Hyper-parameters
input_dim = input_data.shape[1]
output_dim = input_data.shape[1]
latent_dim = 36         # 12 * 3
encoder_layer_sizes = [input_dim, 256, 256, latent_dim]
decoder_layer_sizes = [latent_dim, 256, 256, output_dim]
num_epochs = 100
learning_rate = 0.001
act_fn = "elu"
keep_prob = .2

# model, loss and scheduler
ae = StackedDenoisingAutoEncoder(encoder_layer_sizes, activation=nn.ELU(), final_activation=nn.ELU())
model = DEC_AE(DEC(36, 36, ae.encoder), ae.decoder)

criterion = nn.MSELoss(reduction="mean")
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)

print(model)


DEC_AE(
  (encoder): DEC(
    (encoder): Sequential(
      (0): Sequential(
        (linear): Linear(in_features=63, out_features=256, bias=True)
        (activation): ELU(alpha=1.0)
      )
      (1): Sequential(
        (linear): Linear(in_features=256, out_features=256, bias=True)
        (activation): ELU(alpha=1.0)
      )
      (2): Sequential(
        (linear): Linear(in_features=256, out_features=36, bias=True)
      )
    )
    (assignment): ClusterAssignment()
  )
  (decoder): Sequential(
    (0): Sequential(
      (linear): Linear(in_features=36, out_features=256, bias=True)
      (activation): ELU(alpha=1.0)
    )
    (1): Sequential(
      (linear): Linear(in_features=256, out_features=256, bias=True)
      (activation): ELU(alpha=1.0)
    )
    (2): Sequential(
      (linear): Linear(in_features=256, out_features=63, bias=True)
      (activation): ELU(alpha=1.0)
    )
  )
)


In [22]:
total_step = len(train_loader)
i = 0
n_epochs_no_improve = 5

train_loader_len = float(len(train_loader))
val_loader_len = float(len(val_loader))
test_loader_len = float(len(test_loader))

last_avg_training_loss = 0
min_loss = np.inf
epochs_no_improve = 0
best_model_after_epoch = 0

for epoch in range(num_epochs):
    training_loss = 0
    # training
    for inputs, labels in train_loader:
        # inputs = inputs.to(device)
        # outputs = outputs.to(device)

        pred = model(inputs)
        loss = criterion(pred, labels)
        training_loss+=loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

    last_avg_training_loss = training_loss / train_loader_len
    print ('Epoch [{}/{}], Loss: {:.4f}'
        .format(epoch+1, num_epochs, last_avg_training_loss))

    # early stopping
    with torch.no_grad():
        val_loss = 0
        for inputs, labels in val_loader:
            pred_val = model(inputs)
            loss_val = criterion(pred_val, labels)
            val_loss += loss_val.item()

        val_loss /= val_loader_len
        if min_loss > val_loss:
            min_loss = val_loss
            epochs_no_improve = 0
            best_model_after_epoch = epoch

        else:
            epochs_no_improve+=1
            if epochs_no_improve > n_epochs_no_improve:
                print("Early stopping at Epoch: ", epoch)
                print("last training loss: {:2f}".format(last_avg_training_loss))
                print("achieved best validation loss: {:.4f} after at Epoch {}".format(min_loss, best_model_after_epoch))
                break

# Testing
with torch.no_grad():
    test_loss = 0
    for inputs, labels in test_loader:
        pred_test = model(inputs)
        loss_test = criterion(pred_test, labels)
        test_loss += loss_test.item()

    test_loss /= test_loader_len
    print("Test loss: {:.4f}".format(test_loss))


Epoch [1/100], Loss: 0.1884
Epoch [2/100], Loss: 0.1721
Epoch [3/100], Loss: 0.1721
Epoch [4/100], Loss: 0.1721
Epoch [5/100], Loss: 0.1721
Epoch [6/100], Loss: 0.1721
Epoch [7/100], Loss: 0.1721
Early stopping at Epoch:  6
last training loss: 0.172100
achieved best validation loss: 0.1820 after at Epoch 0
Test loss: 0.1849
